<a href="https://colab.research.google.com/github/ahmedmabrouk24/Computer_vision/blob/main/person_Recognetion_with_logs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tflearn 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 7.6 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=90f7ea878ff280cbacff0640e8a903419ea3ed75eade1e213862fa467611b9dc
  Stored in directory: /root/.cache/pip/wheels/65/9b/15/cb1e6b279c14ed897530d15cfd7da8e3df8a947e593f5cfe59
Successfully built tflearn


In [ ]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import pandas as pd
from tensorflow.keras import layers,models,Sequential
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dense, Conv2D, Activation, Flatten, Dropout, BatchNormalization, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from keras.saving.save import load_model
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tensorflow as tf
import cv2
import random
import numpy as np
import os
import matplotlib.pyplot as plt
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
from PIL import Image
import shutil
import json


Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
TRAIN_DIR_A = '/content/drive/MyDrive/Computer Vision/Dataset_1/personA/Train'
TRAIN_DIR_B = '/content/drive/MyDrive/Computer Vision/Dataset_1/personB/Train'
TRAIN_DIR_C = '/content/drive/MyDrive/Computer Vision/Dataset_1/personC/Train'
TRAIN_DIR_D = '/content/drive/MyDrive/Computer Vision/Dataset_1/personD/Train'
TRAIN_DIR_E = '/content/drive/MyDrive/Computer Vision/Dataset_1/personE/Train'
TRAIN_DIR=[]
TRAIN_DIR.append(TRAIN_DIR_A)
TRAIN_DIR.append(TRAIN_DIR_B)
TRAIN_DIR.append(TRAIN_DIR_C)
TRAIN_DIR.append(TRAIN_DIR_D)
TRAIN_DIR.append(TRAIN_DIR_E)


Test_DIR_A = '/content/drive/MyDrive/Computer Vision/Dataset_1/personA/Test'
Test_DIR_B = '/content/drive/MyDrive/Computer Vision/Dataset_1/personB/Test'
Test_DIR_C = '/content/drive/MyDrive/Computer Vision/Dataset_1/personC/Test'
Test_DIR_D = '/content/drive/MyDrive/Computer Vision/Dataset_1/personD/Test'
Test_DIR_E = '/content/drive/MyDrive/Computer Vision/Dataset_1/personE/Test'
Test_DIR=[]
Test_DIR.append(Test_DIR_A)
Test_DIR.append(Test_DIR_B)
Test_DIR.append(Test_DIR_C)
Test_DIR.append(Test_DIR_D)
Test_DIR.append(Test_DIR_E)


train_images=[]
test_images=[]


IMG_SIZE = 200
num_classes = 5
MODEL_NAME = 'signature_model1'


In [ ]:
def create_train_data():
    training_data = []

    for path_TRAIN_DIR in tqdm(TRAIN_DIR):
       for img in os.listdir(path_TRAIN_DIR):
        if 'csv' in img:
          continue
        path = os.path.join(path_TRAIN_DIR, img)
        img_data = cv2.imread(path,0) 
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img_data),ord(img[6])-ord('A') ])

    random.shuffle(training_data)

    return training_data


In [ ]:
column1=[]
def create_test_data():
    test_data = []

    for path_Test_DIR in tqdm(Test_DIR):
       for img in os.listdir(path_Test_DIR):
        if 'csv' in img:
          continue
        column1.append(img)
        path = os.path.join(path_Test_DIR, img)
        img_data = cv2.imread(path,0) 
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        test_data.append([np.array(img_data),ord(img[6])-ord('A') ])
       

    #random.shuffle(test_data)
    return test_data


In [ ]:
train_images = create_train_data()
X_train = np.array([i[0] for i in train_images]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_train = np.array([i[1] for i in train_images]).reshape(-1,)

test_images = create_test_data()
X_test = np.array([i[0] for i in test_images]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_test = np.array([i[1] for i in test_images]).reshape(-1,)

100%|██████████| 5/5 [00:09<00:00,  1.94s/it]


In [ ]:

model1 = Sequential([
        layers.Conv2D(filters = 16 ,kernel_size = (3, 3), input_shape = (IMG_SIZE, IMG_SIZE, 1), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size = (2,2)),

        layers.Conv2D(filters = 32 ,kernel_size = (5, 5), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),
    
        layers.Conv2D(filters = 32 ,kernel_size = (5, 5), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),
    
        layers.Conv2D(filters = 16 ,kernel_size = (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),
  
        layers.Flatten(),
        layers.Dense(1024, activation='relu'),
        Dropout(0.5),
        layers.Dense(5, activation = 'softmax')
])

model1.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate = 0.0001),
    loss=SparseCategoricalCrossentropy(),
    metrics=['accuracy'])
if (os.path.exists('/content/drive/MyDrive/person_Recognetion.h5')):
      model1 = load_model('/content/drive/MyDrive/person_Recognetion.h5')
else:
      model1.fit(X_train, y_train, epochs = 25)
      model1.save('/content/drive/MyDrive/person_Recognetion.h5')

  

Instructions for updating:
Colocations handled automatically by placer.


Train on 200 samples
Epoch 1/25
200/200 [==============================] - 14s 69ms/sample - loss: 2.3580 - acc: 0.4200
Epoch 2/25
200/200 [==============================] - 13s 65ms/sample - loss: 0.4749 - acc: 0.8350
Epoch 3/25
200/200 [==============================] - 13s 65ms/sample - loss: 0.1435 - acc: 0.9300
Epoch 4/25
200/200 [==============================] - 17s 83ms/sample - loss: 0.1276 - acc: 0.9600
Epoch 5/25
200/200 [==============================] - 14s 72ms/sample - loss: 0.0433 - acc: 0.9850
Epoch 6/25
200/200 [==============================] - 13s 65ms/sample - loss: 0.0208 - acc: 1.0000
Epoch 7/25
200/200 [==============================] - 13s 65ms/sample - loss: 0.0226 - acc: 0.9900
Epoch 8/25
200/200 [==============================] - 13s 65ms/sample - loss: 0.0189 - acc: 0.9950
Epoch 9/25
200/200 [==============================] - 13s 64ms/sample - loss: 0.0218 - acc: 0.9950
Epoch 10/25
200/200 [==============================] - 13s 65ms/sample - loss: 0.0179 - 

In [ ]:
print(model1.evaluate(X_test,y_test))

/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


[0.4842239737510681, 0.85]


In [ ]:
Y_test=model1.predict(X_test)
print(len(X_test))
prediction=[]
for i in Y_test:
  prediction.append(np.argmax(i))
data={'image_name':column1,'label':prediction[:40]}
print(len(column1),len(prediction))
df_person=pd.DataFrame(data)
print(df_person)


/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


40
40 40
        image_name  label
0   personA_10.png      0
1   personA_13.png      0
2    personA_2.png      0
3   personA_20.png      0
4   personA_27.png      0
5   personA_29.png      0
6   personA_30.png      0
7   personA_42.png      0
8   personB_10.png      1
9   personB_13.png      1
10   personB_2.png      1
11  personB_20.png      1
12  personB_27.png      1
13  personB_29.png      1
14  personB_33.png      1
15  personB_42.png      1
16  personC_13.png      2
17  personC_17.png      2
18  personC_19.png      2
19  personC_32.png      2
20  personC_34.png      2
21  personC_39.png      2
22   personC_6.png      2
23   personC_8.png      2
24  personD_14.png      3
25  personD_15.png      3
26  personD_17.png      3
27  personD_21.png      3
28  personD_22.png      3
29  personD_26.png      3
30  personD_31.png      3
31  personD_41.png      3
32  personE_15.png      4
33  personE_17.png      1
34  personE_18.png      1
35  personE_22.png      4
36  personE_27.png      1
37 